In [1]:
from sklearn import datasets
digits = datasets.load_digits()

In [12]:
import numpy as np
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

In [ ]:
from scipy.special import softmax

W = np.random.uniform(-20, 20, size=(10, 64))
b = np. random.uniform(-20,20, size=(10))

probs = softmax(np.maximum(0, np.matmul(W,X[0])+b))
print(probs)

In [6]:
class Node:
    def __init__(self, calculate, predecessors, derives, final=False):
        self.calc = calculate
        self.predecessors = zip(predecessors, derives)
        self.value = None
        self.gradient = None
        self.final = final

def topoSortRec(node, result):
    if node in result:
        return
    for parent in node.predecessors:
        topoSortRec(parent[0], result)
    result.append(node)

def topoSort(nodes):
    result = []
    for node in nodes:
        topoSortRec(node, result)
    return result

def forward(nodes):
    order = topoSort(nodes)
    for node in order:
        node.value = node.calc(*[p[0].value for p in node.predecessors)
    print ("final forward ",order[-1].value)

def backpropagation(nodes):
    order = topoSort(nodes)
    for v in order:
        v.gradient = np.zero()
        if v.final:
            v.gradient = 1
    for v in order[::-1]:
        values = [node.value for node in v.predecessors]
        for [node, derive] in v.predecessors:
            node.gradient += np.matmul(v.gradient, derive(*values))
        

In [7]:
W = Node(lambda : x, [], [])
x = Node(lambda : x, [], [])
a = Node(lambda W,x: np.matmul(W,x), [W,x], [lambda W,x: ])